# Cadence Conversion

## Input formats

nx(<list>)+mx(<list>)+...

where
n,m are positive integers
<lists> are one or more rep counts.
    
*Example*
    
1x(1,2,3) should become [1,2,3]

### Case 0:  Setup

In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import re
from sqlalchemy import (create_engine)


PGURI = os.environ['PG_AZURE_URI']
PGPW = os.environ['PG_AZURE_PW']


datasets = Path('/Users/edmundlskoviak/iCloud/Data Sets/')

## Define Functions


In [45]:

def tokenize(inStr):
    if inStr.find('+') > -1:
        return inStr.split('+')
    else:
        return [inStr]

def parseOperators(inStr):
    outList = []
    mode = 'tile'
    if re.match("BW", inStr) or re.match("max", inStr):
        return inStr

    if inStr.count('r') == 1:
        mode = 'repeat'
        inStr = inStr.replace('r','')

    inStr.replace('*','x')
    
    if inStr.count('x') == 0:
        # basic Set
        mult = 1
        set=inStr
    else:
        (mult, set) = inStr.split('x')

    # need to check to see if set has an operator embedded
    if set.count('-->') == 1:
        start, stop = set.split('-->')
        start = int(re.sub('\(','',start))
        stop = int(re.sub('\)','',stop))
        if stop < start:
            step = -1
            stop -= 1
        else:
            step = 1
            stop += 1
        set = np.arange(start, stop, step)
    else:
        # otherwise, evaluate set
        try:
            set = list(eval(set))
        except TypeError as te:
            set = list(eval(set+','))
        except NameError as ne:
            return set
    if mode == 'tile':
        outList= np.tile(set, int(mult))
    elif mode == 'repeat':
        outList = np.repeat(set, int(mult))

    return outList


# New Approach

The csv data referenced below is an extract from the online database.  Need to parse it and create a table to be scanned into the R processor

In [47]:
df = pd.read_csv( datasets / 'sets.csv')

### Case 1:  nx(<list>)

In [ ]:
for cadence in df['cadence']:
    listFinal = []
    for token in tokenize(cadence):
        listFinal.extend(parseOperators(token))
    print(f"In: {cadence}\t Out: {listFinal}")

In [48]:
for weight in df['weights']:
    listFinal = []
    for token in tokenize(weight):
        listFinal.extend(parseOperators(token))
    print(f"In: {weight}\tOut: {listFinal}")

In: BW	Out: ['B', 'W']
In: 25	Out: [25]
In: 95	Out: [95]
In: [30,35,37.5,37,5,37.5]	Out: [30.0, 35.0, 37.5, 37.0, 5.0, 37.5]
In: [95]	Out: [95]
In: [95,115,115,115]	Out: [95, 115, 115, 115]
In: [95,105,110,110]	Out: [95, 105, 110, 110]
In: [95,110,125,130,135]	Out: [95, 110, 125, 130, 135]
In: 32	Out: [32]
In: 14	Out: [14]
In: 28	Out: [28]
In: [60,60,60,65,70,70]	Out: [60, 60, 60, 65, 70, 70]
In: [12,12,12,12]	Out: [12, 12, 12, 12]
In: [185,190,200,220]	Out: [185, 190, 200, 220]
In: BW	Out: ['B', 'W']
In: 135	Out: [135]
In: 18	Out: [18]
In: 28	Out: [28]
In: [105,115,125,140,150]	Out: [105, 115, 125, 140, 150]
In: 16	Out: [16]
In: 14	Out: [14]
In: [18,18.18,20]	Out: [18.0, 18.18, 20.0]
In: [45,45,45,50,55,55]	Out: [45, 45, 45, 50, 55, 55]
In: [185,185,185,185]	Out: [185, 185, 185, 185]
In: [Hang,Blk,RedBlu,RedBlu]	Out: ['[', 'H', 'a', 'n', 'g', ',', 'B', 'l', 'k', ',', 'R', 'e', 'd', 'B', 'l', 'u', ',', 'R', 'e', 'd', 'B', 'l', 'u', ']']
In: 8	Out: [8]
In: 115	Out: [115]
In: 12	Out: [12

AttributeError: 'float' object has no attribute 'find'

In [ ]:
# superceded
cadences = df['cadence'].to_numpy()

for indata in cadences:
    print(f"{indata}:")
    listCadence = parseSeries(indata)
    print(listCadence)

    for cadence in listCadence:
        print(cadence)
        print(f"{getSeries(cadence)}")

#    print(listCadence)
#    if indata.find('+') > -1:
#        print(indata, indata.split('+'))
#    else:  
#        print(f"In: {indata} out: {getSeries(indata)}")

In [ ]:
row, col = df.shape
print (row, col)

In [ ]:
listL = []
rem = ''
start = 0
while len(helper) > 0:
    stop = helper.find('x')
    if stop == -1:
        listL.append(helper[start:])
        helper = ''
    else:
        listL.append(helper[start:stop-2])
        start = stop-1
        helper = helper[start:]
        break

print(listL, helper)

In [ ]:
def tokenize(inStr):
    if inStr.find('+') > -1:
        return inStr.split('+')
    else:
        return [inStr]

def parseOperators(inStr):
    outList = []
    inStr.replace('*','+')
    if inStr.count('x') == 0:
        # basic Set
        outList=list(eval(inStr))
    
    if inStr.count('x') == 1:
        # in  n x (...)[t | r]  default to t
        mode = 'tile'
        if inStr.count('r') == 1:
            mode = 'repeat'
            inStr = inStr.replace('r','')
        (mult, set) = inStr.split('x')
        if mode == 'tile':
            outList= np.tile(list(eval(set)), int(mult))
        elif mode == 'repeat':
            outList = np.repeat(list(eval(set)), int(mult))

    return outList
 
helper = '2x(5,3,2)+2x(1,3)'
#print(tokenize(helper))
#print(parseOperators(tokenize(helper)[0]))
listFinal = []
for token in tokenize(helper):
    listFinal.extend(parseOperators(token))
    print(parseOperators(token))

print(listFinal)

In [ ]:
import re
i = '(1,)'
re.sub('\(','',i)
#i = i.replace('()','')
#print(int('(1'))

In [ ]:
i = '(1,2,3)'
eval(i)
list(eval(i))
y = (1,2,4)
list(y)